In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
from tavily import TavilyClient
from serpapi import GoogleSearch # Import the SerpAPI library

# Import LangChain and Pydantic libraries
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional

# --- ADDED CODE TO HIDE THE WARNING ---
import warnings
warnings.filterwarnings(
    "ignore",
    message="Received a Pydantic BaseModel V1 schema. This is not supported by method=\"json_schema\""
)
warnings.filterwarnings(
    "ignore",
    message="Received a Pydantic BaseModel V1 schema. This is not supported by method=\"json_schema\". Please use method=\"function_calling\" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method=\"function_calling\""
)
import warnings

# Suppress the specific LangChainDeprecationWarning from IPython/Pydantic
warnings.filterwarnings(
    "ignore",
    message="As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly."
)
#END CODE TO HIDE THE WARNING ---


# looks for the .env file in the same directory as the script 
# The code automatically finds the right directory where the script 
# (or notebook) is running and works across  code editors: VS, Jupyter, and terminal.
#You can move the whole project folder anywhere, and it still works.
# It’s cross-platform (Windows, macOS, Linux). No worries about slashes (\ vs /).
# Works inside VS Code, Jupyter, and the terminal the same way.
script_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()

# load_dotenv(...): Loads environment information from an .env. file
# that you will store in a variables from a .env file.
#script_dir / "SERPAPI_API_KEY.env": Builds the full path to SERPAPI_API_KEY.env 
#inside the script’s directory.
load_dotenv(script_dir / "TAVILY_API_KEY.env")
load_dotenv(script_dir / "OPENAI_API_KEY.env")
load_dotenv(script_dir / "SERPAPI_API_KEY.env") 

# Loading Keys
TAVILY_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPAPI_KEY = os.getenv("SERPAPI_API_KEY") 


# Validate API keys
if not TAVILY_KEY:
    raise ValueError("⚠️ TAVILY_API_KEY not found in environment file")
if not OPENAI_API_KEY:
    raise ValueError("⚠️ OPENAI_API_KEY not found in environment file")
if not SERPAPI_KEY: # Validate SerpAPI key
    raise ValueError("⚠️ SERPAPI_API_KEY not found in environment file")


print(f"TAVILY_KEY loaded: {'Yes' if TAVILY_KEY else 'No'}")
print(f"OPENAI_API_KEY loaded: {'Yes' if OPENAI_API_KEY else 'No'}")
print(f"SERPAPI_KEY loaded: {'Yes' if SERPAPI_KEY else 'No'}")
print("=" * 60)

# Initialize API clients
tavily_client = TavilyClient(TAVILY_KEY)
# SerpAPI does not need a client object, the API key is passed directly to the search object.

## Use schemas (via LangChain and Pydantic structured outputs) to structure flight, hotel, and POI data.
# Define data models
# Each class (Flight, Hotel, CarRental, PointOfInterest, TripPlan) inherits from BaseModel 
# (Pydantic v1 here, via langchain_core.pydantic_v1)
# Field(...) adds validation rules and descriptions that help both LangChain and the LLM understand the schema.
class Flight(BaseModel):
    airline: str = Field(description="The name of the airline.")
    price_usd: str = Field(description="The price in USD.")
    dates: str = Field(description="The flight dates.")
    url: Optional[str] = Field(None, description="URL to the flight details.")

class Hotel(BaseModel):
    name: str = Field(description="The name of the hotel.")
    price_usd: str = Field(description="The price per night in USD.")
    rating: str = Field(description="The star rating of the hotel (e.g., 4 stars).")
    url: Optional[str] = Field(None, description="URL to the hotel details.")

class CarRental(BaseModel):
    company: str = Field(description="The car rental company name.")
    price_usd: str = Field(description="The price per day in USD.")
    url: Optional[str] = Field(None, description="URL to the car rental details.")

class PointOfInterest(BaseModel):
    name: str = Field(description="The name of the point of interest.")
    description: str = Field(description="A brief description.")
    url: Optional[str] = Field(None, description="URL to more information.")

class TripPlan(BaseModel):
    flights: List[Flight] = Field(description="A list of flight options.")
    hotels: List[Hotel] = Field(description="A list of hotel recommendations.")
    car_rentals: List[CarRental] = Field(description="A list of car rental options.")
    points_of_interest: List[PointOfInterest] = Field(description="A list of recommended points of interest.")

# Tavily search functions
def tavily_search(query: str, max_results: int = 3):
    results = tavily_client.search(query=query, max_results=max_results)
    return results

def tavily_search_advanced(query: str, max_results: int = 3):
    results = tavily_client.search(
        query=query,
        search_depth="advanced",
        max_results=max_results,
        include_answer=True
    )
    return results

def serpapi_search(query: str, max_results: int = 5):
    """Conducts a search using SerpAPI and returns a simplified string of results."""
    print(f"  - Searching with SerpAPI for: {query}")
    params = {
        "engine": "google",
        "q": query,
        "api_key": SERPAPI_KEY,
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    
    # Process the results into a single string for the LLM
    output_string = ""
    if "organic_results" in results:
        output_string += f"### SerpAPI Organic Results for '{query}':\n"
        for i, res in enumerate(results["organic_results"]):
            if i >= max_results: break
            output_string += f"Title: {res.get('title', 'N/A')}\n"
            output_string += f"Link: {res.get('link', 'N/A')}\n"
            output_string += f"Snippet: {res.get('snippet', 'N/A')}\n\n"
    
    return output_string


def print_structured_plan(plan: TripPlan):
    """Pretty-print the structured trip plan."""
    print("\n\n=== 🎉 FINAL TRIP PLAN 🎉 ===")
    print("=" * 60)
    
    print("\n✈️  Flights:")
    for f in plan.flights:
        print(f"  - {f.airline}: {f.dates} for {f.price_usd}")
        if f.url: print(f"    URL: {f.url}")

    print("\n🏨 Hotels:")
    for h in plan.hotels:
        print(f"  - {h.name} ({h.rating}): {h.price_usd} per night")
        if h.url: print(f"    URL: {h.url}")

    print("\n🚗 Car Rentals:")
    for c in plan.car_rentals:
        print(f"  - {c.company}: {c.price_usd} per day")
        if c.url: print(f"    URL: {c.url}")
        
    print("\n🗺️ Points of Interest:")
    for p in plan.points_of_interest:
        print(f"  - {p.name}")
        print(f"    {p.description}")
        if p.url: print(f"    URL: {p.url}")

def process_with_llm(all_results: str) -> TripPlan:
    """Takes all search results and formats them into a structured plan."""
    
    # OPEN AI call
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    structured_llm = llm.with_structured_output(schema=TripPlan)

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a world-class travel planner. Take the provided search results and format them into a comprehensive, structured trip plan. Extract as much relevant information as possible, including prices, URLs, and descriptions, and fill in all the fields in the structured output. If a piece of information is missing, leave that field as None."),
        ("human", "Here are the search results for a trip to Cabo San Lucas: {tavily_results}")
    ])
    
    chain = prompt | structured_llm

    print("\n\n⏳ Sending results to GPT-4o mini for structuring...")
    trip_plan_object = chain.invoke({"tavily_results": all_results})
    
    return trip_plan_object

def main():
    all_tavily_results = ""

    # === FLIGHTS: Use SerpAPI for structured flight search ===
    print("\n=== FLIGHTS - SEARCHING WITH SERPAPI ===")
    serpapi_flight_query = "flights to Cabo San Lucas Mexico from San Francisco from November 10 to November 20"
    all_tavily_results += serpapi_search(serpapi_flight_query)

    # === HOTELS: Use Tavily for detailed hotel search ===
    print("\n=== HOTELS SEARCH ===")
    hotel_query = "Best hotels in Cabo San Lucas with prices in USD that are rated 4 stars and above"
    results = tavily_search_advanced(hotel_query)
    all_tavily_results += f"\n\n### HOTELS Results:\n" 
    if "answer" in results:
        all_tavily_results += f"Answer: {results['answer']}\n"
    for r in results.get("results", []):
        all_tavily_results += f"Title: {r.get('title', 'N/A')}\n"
        all_tavily_results += f"URL: {r.get('url', 'N/A')}\n"
        all_tavily_results += f"Content: {r.get('content', '')}\n\n"

    # === CAR RENTALS: Use SerpAPI again for a clean list of vendors ===
    print("\n=== CAR RENTALS SEARCH ===")
    serpapi_car_query = "rental cars in Cabo San Lucas with prices in USD"
    all_tavily_results += serpapi_search(serpapi_car_query)

    # === POI: Use Tavily for more descriptive results ===
    print("\n=== POI SEARCH ===")
    poi_query = "Recommendations for points of interest in Cabo San Lucas"
    results = tavily_search_advanced(poi_query)
    all_tavily_results += f"\n\n### POI Results:\n" 
    if "answer" in results:
        all_tavily_results += f"Answer: {results['answer']}\n"
    for r in results.get("results", []):
        all_tavily_results += f"Title: {r.get('title', 'N/A')}\n"
        all_tavily_results += f"URL: {r.get('url', 'N/A')}\n"
        all_tavily_results += f"Content: {r.get('content', '')}\n\n"


    # Process the combined results with the LLM
    try:
        final_plan = process_with_llm(all_tavily_results)
        print_structured_plan(final_plan)
    except Exception as e:
        print(f"\n❌ An error occurred while processing with the LLM: {e}")

if __name__ == "__main__":
    main()

/Users/seanmcallister/miniconda/envs/ai_env/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


TAVILY_KEY loaded: Yes
OPENAI_API_KEY loaded: Yes
SERPAPI_KEY loaded: Yes

=== FLIGHTS - SEARCHING WITH SERPAPI ===
  - Searching with SerpAPI for: flights to Cabo San Lucas Mexico from San Francisco from November 10 to November 20

=== HOTELS SEARCH ===

=== CAR RENTALS SEARCH ===
  - Searching with SerpAPI for: rental cars in Cabo San Lucas with prices in USD

=== POI SEARCH ===


⏳ Sending results to GPT-4o mini for structuring...


=== 🎉 FINAL TRIP PLAN 🎉 ===

✈️  Flights:
  - United Airlines: November 10 - November 20 for $301
    URL: https://www.united.com/en-us/flights-from-san-francisco-to-san-jose-del-cabo
  - American Airlines: November 10 - November 20 for $301
    URL: https://www.aa.com/en-us/flights-from-san-francisco-to-san-jose-del-cabo
  - Alaska Airlines: November 10 - November 20 for $301
    URL: https://www.alaskaair.com/en/flights-from-san-francisco-to-cabo-san-lucas?srsltid=AfmBOooIxEC01u_k9TcMx7s6V3iAeCkNiooOfbBE2dUHYaKm2np1DC17

🏨 Hotels:
  - Playa Grande Reso